In [2]:
import pandas as pd
import numpy as np
# import os
import nltk
# nltk.data.path.append(r"C:\Users\liufa\nltk_data")

# from nltk.tokenize import word_tokenize
# nltk.download('brown')
import matplotlib.pyplot as plt
import collections
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

# keras
import tensorflow as tf
import keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Lambda,\
    GlobalMaxPooling1D, LSTM, Bidirectional, concatenate, Embedding, multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model


import gc
gc.enable()

In [3]:
raw_data=pd.read_csv(r'C:\Users\liufa\Desktop\Kaggle competition\train.csv')

df = raw_data.rename(columns={'excerpt':'text','target':'y'})
df.head(5)

,id,url_legal,license,text,y,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [4]:
import string
string.punctuation

def remove_punct(text):
    text_nopunct=''.join([char for char in text if char not in string.punctuation])
    return text_nopunct

df['text_no_punct']=df['text'].apply(lambda x: remove_punct(x))

import re
def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens
df['text_tokenized']=df['text_no_punct'].apply(lambda x: tokenize(x.lower()))

import nltk
stopwords=nltk.corpus.stopwords.words('english')
def remove_stopwords(tokenized_list):
    text=[word for word in tokenized_list if word not in stopwords]
    return text
df['text_no_stopwords']=df['text_tokenized'].apply(lambda x: remove_stopwords(x))

wn=nltk.WordNetLemmatizer()
def lemmatizing(tokenized_text):
    text=[wn.lemmatize(word) for word in tokenized_text]
    return text
df['text_lemmatizer']=df['text_no_stopwords'].apply(lambda x: lemmatizing(x))

df['text_clean']=df['text_lemmatizer'].apply(lambda x: ' '.join(x))
df.head()

,id,url_legal,license,text,y,standard_error,text_no_punct,text_tokenized,text_no_stopwords,text_lemmatizer,text_clean
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...","[young, people, returned, ballroom, presented,...","[young, people, returned, ballroom, presented,...",young people returned ballroom presented decid...
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...","[dinner, time, mrs, fayre, somewhat, silent, e...","[dinner, time, mr, fayre, somewhat, silent, ey...",dinner time mr fayre somewhat silent eye resti...
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...","[roger, predicted, snow, departed, quickly, ca...","[roger, predicted, snow, departed, quickly, ca...",roger predicted snow departed quickly came two...
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...","[outside, palace, great, garden, walled, round...","[outside, palace, great, garden, walled, round...",outside palace great garden walled round fille...
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...","[upon, time, three, bears, lived, together, ho...","[upon, time, three, bear, lived, together, hou...",upon time three bear lived together house wood...


In [5]:
df['sentence_len']=df['text'].apply(lambda x: len(x)-x.count(' '))
df['word_len']=df['text_tokenized'].apply(lambda x: len(x))
def count_punct(text):
    ct=sum([1 for char in text if char in string.punctuation])
    return round(ct/(len(text)-text.count(' ')),3)
df['punct_count']=df['text'].apply(lambda x:count_punct(x))
df.head()

,id,url_legal,license,text,y,standard_error,text_no_punct,text_tokenized,text_no_stopwords,text_lemmatizer,text_clean,sentence_len,word_len,punct_count
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...","[young, people, returned, ballroom, presented,...","[young, people, returned, ballroom, presented,...",young people returned ballroom presented decid...,819,179,0.033
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...","[dinner, time, mrs, fayre, somewhat, silent, e...","[dinner, time, mr, fayre, somewhat, silent, ey...",dinner time mr fayre somewhat silent eye resti...,774,169,0.072
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...","[roger, predicted, snow, departed, quickly, ca...","[roger, predicted, snow, departed, quickly, ca...",roger predicted snow departed quickly came two...,747,166,0.063
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...","[outside, palace, great, garden, walled, round...","[outside, palace, great, garden, walled, round...",outside palace great garden walled round fille...,747,164,0.044
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...","[upon, time, three, bears, lived, together, ho...","[upon, time, three, bear, lived, together, hou...",upon time three bear lived together house wood...,577,147,0.055


In [6]:
df.columns

Index(['id', 'url_legal', 'license', 'text', 'y', 'standard_error',
       'text_no_punct', 'text_tokenized', 'text_no_stopwords',
       'text_lemmatizer', 'text_clean', 'sentence_len', 'word_len',
       'punct_count'],
      dtype='object')

In [7]:
# tokenization
X_train, X_test, y_train, y_test = train_test_split(df['text_clean'],df['y'], test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(len(X_train), len(X_train[0]))

# padding
vocab_size = len(tokenizer.word_index) + 1

maxlen = 110

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
print(len(X_train), len(X_train[0]))

# load glove wikipedia embedding
embeddings_dictionary = dict()
glove_file = open(r'D:/datasets/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

embedding_matrix = np.zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        
embedding_matrix.shape

2267 78
2267 110


(22767, 100)

In [8]:
# LSTM model
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
# model.add(LSTM(128,return_sequences=True))
model.add(LSTM(216))
model.add(Dense(21))
model.add(Dense(1))
model.compile(optimizer=Adam(), loss='mse', metrics=['mean_squared_error'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 110, 100)          2276700   
_________________________________________________________________
lstm (LSTM)                  (None, 216)               273888    
_________________________________________________________________
dense (Dense)                (None, 21)                4557      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 22        
Total params: 2,555,167
Trainable params: 278,467
Non-trainable params: 2,276,700
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train, y_train, batch_size=52, epochs=15, verbose=1, validation_split=0.35)

score = model.evaluate(X_test, y_test, verbose=1)
print("Test Scores:", score)

Epoch 1/15
